In [ ]:
import numpy as np

from scipy import signal
from scipy import integrate
from scipy import fftpack

import matplotlib.pyplot as plt

from utils import *

plt.style.use('seaborn-pastel')

## Simulación de edificio de 4 pisos como masas puntuales con rigideces intermedias

In [ ]:
# Edificio de 4 pisos de 20 toneladas cada uno, con rigideces laterales de 57MN/m entre si
k = 57e6 # N/m
m = 20e3 # kg

k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

# Matrices de rigidez y de masa
K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))

print(('K = {}').format(K))
print(('M = {}').format(M))

In [ ]:
# Resuelvo autovectores y autovalores
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)
lamb = np.flip(lamb) # menor a mayor
eigv = np.flip(eigv, axis=1)

In [ ]:
# convierto velocidades angulares a frecuencias
w_modes = np.sqrt(lamb)
f_modes = w_modes/2/np.pi
print(('f = {}').format(np.round(f_modes,3)))

In [ ]:
# Matriz de autovectores (columnas)
print("eigv:")
printMatrix(eigv)

In [ ]:
# Matriz de masas modales
M_modal = np.transpose(eigv)@M@eigv
M_modal[M_modal<1e-3] = np.nan
print("M_modal:")
printMatrix(M_modal)

In [ ]:
# Matriz de rigideces modales
K_modal = np.transpose(eigv)@K@eigv
K_modal[K_modal<1e-3] = np.nan
print("K_modal:")
printMatrix(K_modal)

In [ ]:
# Verifico la validez de las matrices obteniendo las fercuencias modales nuevamente, ahora como w_i = sqrt(k_i/m_i)
w_2 = np.zeros(len(w_modes))
for mode in range(len(w_2)):
    w_2[mode] = np.sqrt(K_modal[mode,mode]/M_modal[mode,mode])
f_2 = w_2/np.pi/2
print(('f2 = {}').format(f_2))

In [ ]:
# Normalizo vectores modales
eigv_norm = np.zeros(eigv.shape)
for mode in range(len(w_modes)):
    eigv_norm[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))
print("eigv_norm:")
printMatrix(eigv_norm)

In [ ]:
# MAC teórico
MAC_theo = get_MAC_matrix(eigv_norm, eigv_norm)
plot_MAC(MAC_theo, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_theo)))

In [ ]:
# Ploteo de modos
plot_modes(eigv_norm)

In [ ]:
animate_modes(eigv_norm, gif_name='theoretical_modes')

## Simulación de aceleración con perfil de ruido blanco

In [ ]:
# Aceleracion maxima en funcion del tiempo
def max_accel(t,t_end):
    a = 0.5*(1-np.cos(2*np.pi*t/t_end))
    return a

In [ ]:
# Simulacion de aceleracion en forma de ruido blanco bajo la curva de aceleracion maxima
t_end = 2400
delta_t = 1/119
t = np.linspace(0, t_end, int(t_end/delta_t))

R = 0.1*np.random.normal(size=len(t)) # Vector de ruido gaussiano
R_mean = np.mean(R)

accel = (R - R_mean)*max_accel(t,t_end) # Aceleracion escalada con ruido
max_accel_vec = max_accel(t,t_end) # Vector con aceleraciones puras

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
fig.add_axes()
ax.plot(t, accel, color='k', marker='')
ax.plot(t, max_accel_vec, color='r')
ax.grid(True, markevery=1)

In [ ]:
# PSD de aceleraciones en la base
nperseg=450
freq, psd = signal.welch(accel, 
                      fs=1./(delta_t), # sample rate
                      window='hamming', # apply a Hanning window before taking the DFT
                      nperseg=nperseg, # compute periodograms of 256-long segments of x
                      noverlap=nperseg//2,
                      detrend='constant',
                      return_onesided=False) #'constant') # detrend x by subtracting the mean)
plt.figure()
plt.semilogy(freq[1:len(psd)//2], psd[1:len(psd)//2])
plt.ylim([np.min(psd[1:]), np.max(psd[1:])])
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.grid()
plt.show()
print(len(psd))

In [ ]:
print('ASD promedio: {:.4f} g²/Hz'.format(np.mean(psd)))
print('Grms para ASD constante = ASD*f_max = {:.3f} g_rms'.format(np.mean(psd)*freq[-1]))

In [ ]:
# calculo de g_rms por integracion
area = integrate.simps(psd, freq)
g_rms = np.sqrt(area)
print('Grms = {:.3f} g_rms'.format(g_rms))

## Simulación de respuesta dinámica del edificio a ruido blanco su base

In [ ]:
xi = 0.025 # fracción de amortiguamiento crítico
r = np.array([1, 1, 1, 1]).reshape(4,1) # vector logico de desplazamientos respecto de la base
print('xi = {}'.format(xi))
print('r = {}^T'.format(r[:,0]))

La solucion del espectro de desplazamientos para cada modo es
<br/>
<center> $Y(\omega) = \frac{\frac{\iota}{m_i} U_g(\omega)}{\omega_i^2 - \omega^2 + 2i \xi_i \omega_i \omega}$ </center>
<br/>
con
<center> $\iota = \Phi^T M r$. </center>
<br/>
Y la aceleración es
<center> $\ddot{Y}(\omega) = \omega^2 Y(\omega)$ </center>

In [ ]:
U_g = np.copy(psd)
w = 2*np.pi*freq

In [ ]:
I = np.transpose(eigv)@M@r
print(I)
m_modal = M_modal.diagonal() 
Y = np.zeros((len(w), len(w_modes)), dtype=complex)
ddotY = np.copy(Y) 
for mode in range(Y.shape[1]):
    C = I[mode] / (m_modal[mode]*(w_modes[mode]**2 - w**2 + 2*1j*xi*w_modes[mode]*w))
    Y[:,mode] = C*U_g
    ddotY[:,mode] = (w**2)*Y[:,mode]

In [ ]:
f = w/2/np.pi
plt.figure()
lgnd  = ['modo {}'.format(mode+1) for mode in range(Y.shape[1])]
plt.semilogy(f, abs(Y))
plt.title('Desplazamientos modales')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.legend(lgnd)
plt.show()

In [ ]:
plt.figure()
lgnd  = ['modo {}'.format(mode+1) for mode in range(Y.shape[1])]
plt.semilogy(f, abs(ddotY)/9.81)
plt.title('Aceleraciones modales')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de Aceleraciones [g/Hz]')
plt.legend(lgnd)
plt.show()
print(np.max(abs(ddotY))/9.81)

In [ ]:
freq_max = [abs(f[i]) for i in ddotY.argmax(axis=0)]
print(('freqs from max response = {}').format(np.round(freq_max, 3)))
print('errors in % = {}'.format(np.round(100*(f_modes - freq_max) / f_modes, 2)))

### Respuesta en cada piso

In [ ]:
# Espectro de respuesta en frecuencia
X = np.copy(Y)
X = (eigv @ Y.T).T # Desplazamientos en los grados de libertad
ddotX = (w**2 * X.T).T # Aceleraciones

# Respuesta en el tiempo
X_time = fftpack.ifft(X, axis=0) # Obtengo respuestas en el tiempo mediante transformada inversa

In [ ]:
plt.figure()
lgnd  = ['piso {}'.format(mode+1) for mode in range(X.shape[1])]
plt.semilogy(f, abs(X))
plt.title('Desplazamientos por piso')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.legend(lgnd)
plt.show()

## Peak-picking

In [ ]:
X_psd = np.copy(X)
for piso in range(X.shape[1]):
    X_psd[:, piso] = X[:, piso] * np.conj(X[:, piso])

In [ ]:
# Average Normalized PSD (ANPSD)
ANPSD = np.zeros(X.shape[0], dtype=complex)
NPSD = np.copy(X_psd)
for piso in range(X.shape[1]):
    NPSD[:, piso] = X_psd[:, piso] / np.sum(X_psd[:, piso])
ANPSD = np.sum(NPSD, axis=1)
mode_ind_pp = np.array([m for m in signal.argrelmax(ANPSD, order=5)]).flatten()

In [ ]:
plt.figure()
plt.semilogy(f, abs(ANPSD), label='ANPSD')
plt.scatter(f[mode_ind], abs(ANPSD[mode_ind]), label='modes', color='r')
plt.title('Average Normalized PSD')
plt.ylim([1E-6, 10])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.legend()
plt.show()

In [ ]:
print('Frecuencias modales segun Peak-picking')
for mode in range(len(w_modes)):
    idx = mode_ind[mode]
    print('f_{} = {:.3f} Hz ({:.2f}% error)'.format(mode,
                                             f[idx],
                                             100*(f[idx] - f_modes[mode])/ f_modes[mode]))

### Formas modales

In [ ]:
modes_pp = np.copy(eigv) 
for mode in range(modes_pp.shape[1]):
    idx = mode_ind[mode]
    for dof in range(X.shape[1]):
        cross_power = X[idx, dof] * np.conj(X[idx, 0])
        ang = abs(np.angle(cross_power, deg=True))
        if 0 <= ang <= 60:
            sign = 1
        elif 120 <= ang <=180:
            sign = -1
        else:
            sign = 0
        modes_pp[dof, mode] = sign * abs(X[idx, dof]) / abs(X[idx, 0])
        
# Normalizacion
for col in range(modes_pp.shape[1]):
    modes_pp[:,col] = modes_pp[:,col]/max(abs(modes_pp[:,col]))
printMatrix(modes_pp)

In [ ]:
# MAC para Peak-picking
MAC_pp = get_MAC_matrix(eigv_norm, modes_pp)
plot_MAC(MAC_pp, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_pp)))

In [ ]:
# Ploteo de modos
plot_modes(modes_pp)

## Frequency Domain Decomposition

In [ ]:
# Respuesta en el tiempo
ddotX_time = fftpack.ifft(ddotX, axis=0) # Obtengo respuestas en el tiempo mediante transformada inversa

# Matriz de densidades espectrales cruzadas de la respuesta.
csd_nperseg = 450
S_xx = np.zeros((csd_nperseg, ddotX_time.shape[1], ddotX_time.shape[1]), dtype=complex)
for piso1 in range(ddotX.shape[1]):
    for piso2 in range (ddotX.shape[1]):
        ff, S_xx[:, piso1, piso2] = signal.csd(ddotX_time[:, piso1], ddotX_time[:, piso2],
                                               fs=1./(delta_t),
                                               window='hanning',
                                               nperseg=csd_nperseg,
                                               detrend='constant',
                                               axis=0,
                                               return_onesided=False)

In [ ]:
# Extraigo frecuencias y formas de modo por SVD
u, s, vh = np.linalg.svd(S_xx)
    
# Maximos del primer valor singular
mode_ind_fdd = np.array([m for m in signal.argrelmax(s[:, 0], order=5)]).flatten()

In [ ]:
plt.semilogy(ff[:len(ff)//2], s[0, :len(ff)//2])
plt.semilogy(ff[:len(ff)//2], s[1, :len(ff)//2])
plt.semilogy(ff[:len(ff)//2], s[2, :len(ff)//2])
plt.semilogy(ff[:len(ff)//2], s[3, :len(ff)//2])
plt.ylim([np.min(s[0, :len(ff)//2]), np.max(s[0, :len(ff)//2])])
plt.scatter(ff[mode_ind[:len(mode_ind)//2]], abs(s[0, mode_ind[:len(mode_ind)//2]]), color='r')

In [ ]:
ff[mode_ind[:4]]

In [ ]:
# Formas modales
modes_fdd = np.copy(eigv)
for idx in range(modes_fdd.shape[1]):
    modes_fdd[:, idx] = u[:, 0, mode_ind[idx]]

# Normalizo
for col in range(modes_fdd.shape[1]):
    modes_fdd[:,col] = modes_fdd[:,col]/max(abs(modes_fdd[:,col]))
printMatrix(modes_fdd)

In [ ]:
# MAC para FDD
MAC_fdd = get_MAC_matrix(eigv_norm, modes_fdd)
plot_MAC(MAC_fdd, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_fdd)))

In [ ]:
# Ploteo de modos
plot_modes(modes_fdd)